# 隐语SecretFlow金融风控全链路能力展示

> This tutorial is only available in Chinese.

> Last updated: May 12, 2023
>
> 请使用v0.8.3或以上版本的隐语进行实验。
>
> 以下代码仅作为示例，请勿在生产环境直接使用。

本次实验将会展示如何使用隐语进行在风控领域常用的Logistic Regeression模型和XGB模型的模型研发工作。

隐语接下来将会开放模型部署和在线/离线模型预测功能，敬请期待。

## 实验目标

在本次实验中，我们将会利用一个开源数据集训练一个金融风控场景常用的线性回归和XGB模型。在此过程中将包含以下步骤：

- 样本对齐
- 特征预处理
- 数据分析
- 模型训练
- 模型预测
- 模型评估

请依次执行所有步骤确保实验可以顺利完成。

## 实验前置工作

### 初始化隐语框架

在本次实验中，我们将会包含两个节点：**alice** 和 **bob** . 在真实业务场景，他们将会代表两个不同实体，他们之间的原始数据不被允许直接相互传输，但是他们的原始数据将会被一起用以研发一个模型。

在下面的代码中，我们建立了一个 **SecretFlow Cluster**, 基于 **alice** 和 **bob** 两个节点，我们还创建了三个device：

- alice: PYU device, 负责在alice侧的本地计算，计算输入、计算过程和计算结果仅alice可见
- bob: PYU device, 负责在bob侧的本地计算，计算输入、计算过程和计算结果仅bob可见
- spu: SPU device, 负责alice和bob之间的密态计算，计算输入和计算结果为密态，由alice和bob各掌握一个分片，计算过程为MPC计算，由alice和bob各自的SPU Runtime一起执行。

>  如果你尚未理解以上的一些概念，比如SPU设备，请参考这篇[文档](../developer/design/architecture.md).


In [1]:
import secretflow as sf

sf.shutdown()
sf.init(['alice', 'bob'], address='local')
alice, bob = sf.PYU('alice'), sf.PYU('bob')
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))


2023-06-17 19:38:13,943	INFO worker.py:1538 -- Started a local Ray instance.


在上面的log中，你应该发现，在**spu**的创建过程中，alice和bob两边都各有一个 **SPURuntime** 被建立并互相创建连接。

### 数据集

本次实验我们采用的原始数据是来自UCI的[Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/bank+marketing). 这个数据集汇集了一家葡萄牙银行机构电话营销的结果。

我们添加了**uid**这一列用于接下来隐私求交的实验。

我们首先看一下数据集所包含的信息。


In [2]:
import pandas as pd

# secretflow.utils.simulation.datasets contains mirrors of some popular open dataset.
from secretflow.utils.simulation.datasets import dataset

df = pd.read_csv(dataset('bank_marketing_full'), sep=';')
df['uid'] = df.index + 1

df


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,uid
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,1
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,2
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,3
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,4
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes,45207
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes,45208
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes,45209
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no,45210


该数据集包含了45211个样本，每一个样本代表了一个目标客户。

每个样本包含16个feature，我们这里简单描述一下这个数据集所有的feature。


| feature | 描述 | 取值 |
| :-----| :---- | :---- |
| uid | 客户编码 | 数字 |
| age | 年龄 | 数字 |
| job | 工作类型 |  'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown' |
| marital | 婚姻状况 | 'divorced','married','single','unknown' |
| education | 教育状况 | 'tertiary', 'secondary', 'unknown', 'primary' |
| default | 是否有不良信用记录 | 'no','yes','unknown' |
| housing | 是否有房贷 |  'no','yes','unknown' |
| loan | 是否有个人贷款 | 'no','yes','unknown' |
| contact | 联系方式 | 'cellular','telephone' |
| month | 上次联系月份 | 'jan', 'feb', 'mar', ..., 'nov', 'dec' |
| day | 上次联系月日 |数字|
| duration | 上次沟通时间 | 数字 |
| campaign | 本次活动已经沟通的次数 | 数字 |
| pdays | 距离上次沟通经过的天数 | 数字 |
| previous | 在本次活动之前已经沟通的次数 | 数字 |
| poutcome | 之前活动的结果 | 'unknown', 'failure', 'other', 'success' | 




每个样本的label - y表示对于目标客户的营销结果（是否签订了定额存款合同），取值是'yes','no'。

我们假定以上16个feature由两个机构分别掌握，具体如下。

- alice: age, job, marital, education, default, balance, housing, loan
- bob: contact, day, month, duration, campaign, pdays, previous, poutcome, y


在真实业务场景中, alice和bob所掌握的数据可能是没有对齐的，为了模拟这种情况，我们将数据集shuffle之后，再随机各取90%来模拟这个状况。

In [3]:
import numpy as np

df_alice = df.iloc[:, np.r_[0:8, -1]].sample(frac=0.9)

df_alice


,age,job,marital,education,default,balance,housing,loan,uid
11052,38,blue-collar,married,primary,no,7944,no,no,11053
22112,37,management,single,tertiary,no,102,yes,no,22113
35603,52,management,married,tertiary,no,249,yes,yes,35604
5197,37,management,married,tertiary,no,412,yes,no,5198
29993,33,entrepreneur,single,tertiary,no,138,no,no,29994
...,...,...,...,...,...,...,...,...,...
15183,49,blue-collar,married,secondary,no,388,no,no,15184
559,42,technician,married,secondary,no,315,yes,no,560
37222,32,management,single,secondary,no,-119,yes,no,37223
9054,51,services,married,secondary,no,540,yes,no,9055


In [4]:
df_bob = df.iloc[:, 8:].sample(frac=0.9)

df_bob


,contact,day,month,duration,campaign,pdays,previous,poutcome,y,uid
5552,unknown,23,may,937,3,-1,0,unknown,no,5553
6910,unknown,28,may,117,2,-1,0,unknown,no,6911
6042,unknown,26,may,129,3,-1,0,unknown,no,6043
42428,cellular,7,dec,143,1,283,3,failure,no,42429
26674,cellular,20,nov,768,3,-1,0,unknown,yes,26675
...,...,...,...,...,...,...,...,...,...,...
21756,cellular,19,aug,599,3,-1,0,unknown,no,21757
35794,cellular,8,may,51,1,-1,0,unknown,no,35795
15669,telephone,21,jul,158,1,-1,0,unknown,no,15670
4234,unknown,19,may,285,1,-1,0,unknown,no,4235


我们这里将df_alice和df_bob保存为文件，作为alice和bob两方的原始输入。

至此，我们完成了所有实验准备工作。

In [5]:
import tempfile

_, alice_path = tempfile.mkstemp()
_, bob_path = tempfile.mkstemp()
df_alice.reset_index(drop=True).to_csv(alice_path, index=False)
df_bob.reset_index(drop=True).to_csv(bob_path, index=False)


## 样本对齐（隐私求交）

显然，第一步我们需要将两边的数据对齐。
隐私求交（[Private Set Intersection](https://en.wikipedia.org/wiki/Private_set_intersection))是一种密码学方法，可以获取两个集合的交集，而不泄露任何其他信息。
在隐语中，SPU设备支持三种隐私求交算法:

- [ECDH](https://ieeexplore.ieee.org/document/6234849/)：半诚实模型, 基于公钥密码学，原本适用于小数据集，但是隐语优化后已经能支持10亿量级的数据。
- [KKRT](https://eprint.iacr.org/2016/799.pdf)：半诚实模型, 基于布谷鸟哈希（Cuckoo Hashing）以及高效不经意传输扩展（OT Extension），适用于大数据集（比如千万数据集）。
- [BC22PCG](https://eprint.iacr.org/2022/334)：半诚实模型, 基于随机相关函数生成器，适用于大数据集。

由于我们这里的数据集较小，我们这里采用的是ECDH方法。


### 方式一：将隐私求交结果保存至文件

在一些应用场景场景中，alice和bob可能在隐私求交之后将结果直接保存至文件中，之后再进行后续操作。这个时候，请调用**psi_csv**接口。

在以下代码中，我们分别制定了两边需要求交的key以及输入和输出路径。

我们需要指定双方的输入文件和输出文件路径。对于ECDH来说，由于双方的地位是平等的，receiver并没有实际含义，你可以任意指定。我们需要设定正确的protocol。sort设为true之后，join的结果将会被排序。

> 请阅读 psi_csv 的文档。

In [6]:
_, alice_psi_path = tempfile.mkstemp()
_, bob_psi_path = tempfile.mkstemp()

spu.psi_csv(
    key="uid",
    input_path={alice: alice_path, bob: bob_path},
    output_path={alice: alice_psi_path, bob: bob_psi_path},
    receiver="alice",
    protocol="ECDH_PSI_2PC",
    sort=True,
)


(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:41:15.276 [info] [bucket_psi.cc:Init:228] bucket size set to 1048576
(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:41:15.276 [info] [bucket_psi.cc:Run:97] Begin sanity check for input file: /tmp/tmpwdamb4r6, precheck_switch:true
(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:41:15.292 [info] [csv_checker.cc:CsvChecker:121] Executing duplicated scripts: LC_ALL=C sort --buffer-size=1G --temporary-directory=/tmp --stable selected-keys.1687002075277002203 | LC_ALL=C uniq -d > duplicate-keys.1687002075277002203
(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:41:15.305 [info] [bucket_psi.cc:Run:115] End sanity check for input file: /tmp/tmpwdamb4r6, size=40690
(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:41:15.305 [info] [bucket_psi.cc:RunPsi:419] Run psi protocol=1, self_items_count=40690
(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:4

[{'party': 'alice', 'original_count': 40690, 'intersection_count': 36627},
 {'party': 'bob', 'original_count': 40690, 'intersection_count': 36627}]

(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:41:19.035 [info] [utils.cc:MultiKeySort:90] Finished sort scripts: tail -n +2 /tmp/tmp-sort-in-1687002078996801248 | LC_ALL=C sort --buffer-size=3G --parallel=8 --temporary-directory=./ --stable --field-separator=, --key=9,9 >>/tmp/tmp-sort-out-1687002078996801248, ret=0
(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:41:19.035 [info] [bucket_psi.cc:Run:216] End post filtering, in=/tmp/tmpwdamb4r6, out=/tmp/tmpzvs3e1bk
(SPURuntime(device_id=None, party=bob) pid=8796) 2023-06-17 19:41:19.035 [info] [utils.cc:MultiKeySort:90] Finished sort scripts: tail -n +2 /tmp/tmp-sort-in-1687002078997320675 | LC_ALL=C sort --buffer-size=3G --parallel=8 --temporary-directory=./ --stable --field-separator=, --key=10,10 >>/tmp/tmp-sort-out-1687002078997320675, ret=0
(SPURuntime(device_id=None, party=bob) pid=8796) 2023-06-17 19:41:19.036 [info] [bucket_psi.cc:Run:216] End post filtering, in=/tmp/tmpfvyv7vog, out=/tmp/tmpy

### 方式二：将求交结果保存至VDataFrame

VDataFrame是隐语中保存垂直切分数据的数据结构，在接下来的任务中，我们将会不断使用VDataFrame的数据结构。

由于在本次实验中，经过隐私求交之后，我们还有后续操作，所以我们在这里使用 **data.vertical.read_csv** 来将原始数据隐私求交之后的结果直接转化为VDataFrame。

> 请阅读data.vertical.read_csv的文档。很多参数和psi_csv是一致的，这里不再赘述。

In [7]:
from secretflow.data.vertical import read_csv as v_read_csv

vdf = v_read_csv(
    {alice: alice_path, bob: bob_path},
    spu=spu,
    keys="uid",
    drop_keys="uid",
    psi_protocl="ECDH_PSI_2PC",
)
vdf.columns


(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:41:31.059 [info] [bucket_psi.cc:Init:228] bucket size set to 1048576
(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:41:31.059 [info] [bucket_psi.cc:Run:97] Begin sanity check for input file: /tmp/tmpwdamb4r6, precheck_switch:true
(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:41:31.075 [info] [csv_checker.cc:CsvChecker:121] Executing duplicated scripts: LC_ALL=C sort --buffer-size=1G --temporary-directory=/tmp --stable selected-keys.1687002091060169427 | LC_ALL=C uniq -d > duplicate-keys.1687002091060169427
(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:41:31.089 [info] [bucket_psi.cc:Run:115] End sanity check for input file: /tmp/tmpwdamb4r6, size=40690
(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:41:31.089 [info] [bucket_psi.cc:RunPsi:419] Run psi protocol=1, self_items_count=40690
(SPURuntime(device_id=None, party=alice) pid=8795) 2023-06-17 19:4

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

### 更多

我们在这里展示的是两方单键的隐私求交，隐语也支持三方和多键的隐私求交技术，想要了解更多信息，你可以：

- 阅读这篇[文档](https://www.secretflow.org.cn/docs/spu/en/development/psi.html)了解隐语SPU的隐私求交能力。
- 阅读该[教程](./PSI_On_SPU.ipynb)了解使用的例子。

## 特征预处理

一般情况下，我们都需要对用于建模的数据进行预处理，合理的预处理对模型训练效果非常关键。

在开始特征预处理之前，我们先使用 **stats.table_statistics.table_statistics** 来查看一下特征总体情况，我们会在后面专门讨论全表统计模块。

In [8]:
from secretflow.stats.table_statistics import table_statistics

pd.set_option('display.max_rows', None)
data_stats = table_statistics(vdf)
data_stats

(_run pid=8491) /home/zhanggong/anaconda3/envs/sf/lib/python3.8/site-packages/secretflow/device/device/pyu.py:154: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
(_run pid=8491)   return fn(*args, **kwargs)
(_run pid=8493) /home/zhanggong/anaconda3/envs/sf/lib/python3.8/site-packages/secretflow/device/device/pyu.py:154: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
(_run pid=8493)   return fn(*args, **kwargs)
(_run pid=8493) /home/zhanggong/anaconda3/envs/sf/lib/python3.8/site-packages/secretflow/device/device/pyu.py:154: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default 

,datatype,total_count,count,count_na,min,max,mean,var,std,sem,...,moment_2,moment_3,moment_4,central_moment_2,central_moment_3,central_moment_4,sum,sum_2,sum_3,sum_4
age,int64,36627,36627,0,18.0,95.0,40.945477,1.128962e+02,10.625262,0.055519,...,1.789425e+03,8.334015e+04,4.124033e+06,1.128931e+02,8.263566e+02,4.231767e+04,1499710.0,6.554128e+07,3.052500e+09,1.510510e+11
job,object,36627,36627,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
marital,object,36627,36627,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
education,object,36627,36627,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
default,object,36627,36627,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
balance,int64,36627,36627,0,-6847.0,98417.0,1359.496628,9.050888e+06,3008.469364,15.719724,...,1.089887e+07,2.545282e+11,3.947053e+15,9.050641e+06,2.151026e+11,1.022811e+16,49794283.0,3.991930e+11,9.322603e+15,-3.005232e+18
housing,object,36627,36627,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan,object,36627,36627,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
contact,object,36627,36627,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
day,int64,36627,36627,0,1.0,31.0,15.832009,6.951049e+01,8.337295,0.043564,...,3.201611e+02,7.321293e+03,1.799951e+05,6.950860e+01,5.157828e+01,9.367070e+03,579879.0,1.172654e+07,2.681570e+08,6.592682e+09


In [9]:
pd.reset_option('display.max_rows')

在接下来，我们将会展示隐语以下特征预处理能力：

- 值替换
- 缺失值填充
- WOE分组/分箱转换
- one-hot编码
- 标准化

### 值替换

我们先对以下特征做值替换：

| feature | 描述 | 取值和值替换规则 |
| :-----| :---- | :---- |
| education | 教育状况 | 'tertiary' -> 3, 'secondary' -> 2, 'unknown' -> 0, 'primary' -> 1 |
| default | 是否有不良信用记录 | 'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| housing | 是否有房贷 |  'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| loan | 是否有个人贷款 | 'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| month | 上次联系月份 | 'jan' -> 1, 'feb' -> 2, 'mar' -> 3, ..., 'nov' -> 11, 'dec' ->12 |
| y | label | 'yes' -> 1,'no' -> 0 |


替换完之后，我们使用 **sf.reveal** 来查看效果，请注意在生产中，**sf.reveal** 将会直接泄露数据，需要严格限制和进行审计。

> 在生产中，请严格限制**sf.reveal**的使用。

In [10]:
vdf['education'] = vdf['education'].replace(
    {'tertiary': 3, 'secondary': 2, 'primary': 1, 'unknown': np.NaN}
)

vdf['default'] = vdf['default'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['housing'] = vdf['housing'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['loan'] = vdf['loan'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['month'] = vdf['month'].replace(
    {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr': 4,
        'may': 5,
        'jun': 6,
        'jul': 7,
        'aug': 8,
        'sep': 9,
        'oct': 10,
        'nov': 11,
        'dec': 12,
    }
)

vdf['y'] = vdf['y'].replace(
    {
        'no': 0,
        'yes': 1,
    }
)

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


       age            job  marital  education  default  balance  housing  loan
0       58     management  married        3.0        0     2143        1     0
1       43     technician   single        2.0        0      593        1     0
2       46     management  married        3.0        0      229        1     0
3       33     technician   single        2.0        0       56        0     0
4       38         admin.  married        1.0        0     1487        0     0
...    ...            ...      ...        ...      ...      ...      ...   ...
36622   36  self-employed   single        3.0        0     4844        0     0
36623   49     technician  married        2.0        0      378        0     0
36624   52   entrepreneur  married        3.0        0     1115        1     0
36625   40    blue-collar  married        1.0        0       48        0     0
36626   46       services  married        3.0        0      474        0     0

[36627 rows x 8 columns]
       contact  day  month

#### 安全性讨论

值替换操作由数据所有者的PYU Device执行，不会泄露数据。

### 缺失值填充

接下来我们对缺失值进行填充。我们在这里均填充了众数，其他可选的策略还包括平均数、中位数等。

其他可能的处理方法包括删除缺省的行, 或者可以使用数据完整的行作为训练集，以此来预测缺失值。

替换完之后，我们使用 **sf.reveal** 来查看效果。

In [11]:
vdf["education"] = vdf["education"].fillna(vdf["education"].mode())
vdf["default"] = vdf["default"].fillna(vdf["default"].mode())
vdf["housing"] = vdf["housing"].fillna(vdf["housing"].mode())
vdf["loan"] = vdf["loan"].fillna(vdf["loan"].mode())

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


       age            job  marital  education  default  balance  housing  loan
0       58     management  married        3.0        0     2143        1     0
1       43     technician   single        2.0        0      593        1     0
2       46     management  married        3.0        0      229        1     0
3       33     technician   single        2.0        0       56        0     0
4       38         admin.  married        1.0        0     1487        0     0
...    ...            ...      ...        ...      ...      ...      ...   ...
36622   36  self-employed   single        3.0        0     4844        0     0
36623   49     technician  married        2.0        0      378        0     0
36624   52   entrepreneur  married        3.0        0     1115        1     0
36625   40    blue-collar  married        1.0        0       48        0     0
36626   46       services  married        3.0        0      474        0     0

[36627 rows x 8 columns]
       contact  day  month

#### 安全性讨论

所填充的缺失值由属于数据所有者的PYU Device执行，并在接下来的缺失值操作中由数据所有者的PYU Device使用，不会泄露数据。

### woe分箱

woe分箱用于将连续值替换为离散值。

将连续型特征离散化的一个好处是可以有效地克服数据中隐藏的缺陷： 使模型结果更加稳定。例如，数据中的极端值是影响模型效果的一个重要因素。极端值导致模型参数过高或过低，或导致模型被虚假现象"迷惑"，把原来不存在的关系作为重要模式来学习。而离散化可以有效地减弱极端值和异常值的影响。

变量duration的75%分位数远小于最大值，而且该变量的标准差相对也比较大。因此需要对变量duration进行离散化。

In [12]:
from secretflow.preprocessing.binning.vert_woe_binning import VertWoeBinning
from secretflow.preprocessing.binning.vert_woe_substitution import VertWOESubstitution

binning = VertWoeBinning(spu)
woe_rules = binning.binning(
    vdf,
    binning_method="chimerge",
    bin_num=4,
    bin_names={alice: [], bob: ["duration"]},
    label_name="y",
)

woe_sub = VertWOESubstitution()
vdf = woe_sub.substitution(vdf, woe_rules)

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


INFO:root:Create proxy actor <class 'secretflow.preprocessing.binning.vert_woe_binning_pyu.VertWoeBinningPyuWorker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.preprocessing.binning.vert_woe_binning_pyu.VertWoeBinningPyuWorker'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.preprocessing.binning.vert_woe_substitution.VertWOESubstitutionPyuWorker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.preprocessing.binning.vert_woe_substitution.VertWOESubstitutionPyuWorker'> with party bob.
(_run pid=8491) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=8491) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=8491) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
(_run pid=8491) INFO:jax._src.xla_b

       age            job  marital  education  default  balance  housing  loan
0       58     management  married        3.0        0     2143        1     0
1       43     technician   single        2.0        0      593        1     0
2       46     management  married        3.0        0      229        1     0
3       33     technician   single        2.0        0       56        0     0
4       38         admin.  married        1.0        0     1487        0     0
...    ...            ...      ...        ...      ...      ...      ...   ...
36622   36  self-employed   single        3.0        0     4844        0     0
36623   49     technician  married        2.0        0      378        0     0
36624   52   entrepreneur  married        3.0        0     1115        1     0
36625   40    blue-collar  married        1.0        0       48        0     0
36626   46       services  married        3.0        0      474        0     0

[36627 rows x 8 columns]
       contact  day  month

#### 安全性讨论

woe分桶需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

### One Hot编码

one-hot编码适用于将类型编码转化为数值编码。 对于job、marital等特征我们需要one-hot编码。

In [13]:
from secretflow.preprocessing.encoder import OneHotEncoder

encoder = OneHotEncoder()
# for vif and correlation only
vdf_hat = vdf.drop(columns=["job", "marital", "contact", "month", "day", "poutcome"])

tranformed_df = encoder.fit_transform(vdf['job'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['marital'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['contact'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['month'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['day'])
vdf[tranformed_df.dtypes.index] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['poutcome'])
vdf[tranformed_df.dtypes.index] = tranformed_df

vdf = vdf.drop(columns=["job", "marital", "contact", "month", "day", "poutcome"])

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


       age  education  default  balance  housing  loan  job_admin.  \
0       58        3.0        0     2143        1     0         0.0   
1       43        2.0        0      593        1     0         0.0   
2       46        3.0        0      229        1     0         0.0   
3       33        2.0        0       56        0     0         0.0   
4       38        1.0        0     1487        0     0         1.0   
...    ...        ...      ...      ...      ...   ...         ...   
36622   36        3.0        0     4844        0     0         0.0   
36623   49        2.0        0      378        0     0         0.0   
36624   52        3.0        0     1115        1     0         0.0   
36625   40        1.0        0       48        0     0         0.0   
36626   46        3.0        0      474        0     0         0.0   

       job_blue-collar  job_entrepreneur  job_housemaid  ...  job_retired  \
0                  0.0               0.0            0.0  ...          0.0   
1    

#### 安全性讨论

one-hot编码操作由数据所有者的PYU Device执行，不会泄露数据。

### 标准化 
特征之间数值差距太大会使得模型收敛困难，我们一般先对数值进行标准化。

In [14]:
from secretflow.preprocessing import StandardScaler

X = vdf.drop(columns=['y'])
y = vdf['y']
scaler = StandardScaler()
X = scaler.fit_transform(X)
vdf[X.columns] = X
print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


(_run pid=8491) /home/zhanggong/anaconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=8491)   warnings.warn(
(_run pid=8495) /home/zhanggong/anaconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=8495)   warnings.warn(


            age  education   default   balance   housing      loan  \
0      1.605114   1.311478 -0.134309  0.260436  0.892122 -0.435047   
1      0.193365  -0.221048 -0.134309 -0.254783  0.892122 -0.435047   
2      0.475715   1.311478 -0.134309 -0.375776  0.892122 -0.435047   
3     -0.747801  -0.221048 -0.134309 -0.433282 -1.120923 -0.435047   
4     -0.277218  -1.753575 -0.134309  0.042382 -1.120923 -0.435047   
...         ...        ...       ...       ...       ...       ...   
36622 -0.465452   1.311478 -0.134309  1.158247 -1.120923 -0.435047   
36623  0.758064  -0.221048 -0.134309 -0.326249 -1.120923 -0.435047   
36624  1.040414   1.311478 -0.134309 -0.081271  0.892122 -0.435047   
36625 -0.088985  -1.753575 -0.134309 -0.435941 -1.120923 -0.435047   
36626  0.475715   1.311478 -0.134309 -0.294339 -1.120923 -0.435047   

       job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  ...  \
0       -0.358292        -0.524872         -0.184203       -0.16676  ...   
1      

#### 安全性讨论

标准化操作由数据所有者的PYU Device执行，不会泄露数据。

### 更多

隐语还支持其他更多的特征预处理能力，请参考这篇[文档](./data_preprocessing_with_data_frame.ipynb).

至此，我们已经完成了所有特征预处理工作。

> 本文主要目的是为了展示隐语的预处理能力，本文对于数据预处理方法的使用可能是有争议的，敬请谅解。

## 数据分析

在建模之前，我们有必要分析一下我们所使用的数据，以便确认是否需要重复特征预处理的过程。

下面我们将会展示隐语以下数据分析能力:

- 全表统计
- 相关系数矩阵
- VIF指标计算


### 全表统计

我们提供了类似于 **pd.DataFrame.describe** 来展示所有特征的基本统计信息。

> 在特征预处理的过程中，你可以不断调用全表统计来关注预处理效果。

In [15]:
from secretflow.stats.table_statistics import table_statistics

pd.set_option('display.max_rows', None)
data_stats = table_statistics(vdf)
data_stats


,datatype,total_count,count,count_na,min,max,mean,var,std,sem,...,moment_2,moment_3,moment_4,central_moment_2,central_moment_3,central_moment_4,sum,sum_2,sum_3,sum_4
age,float64,36627,36627,0,-2.159551,5.087428,2.390009e-16,1.000027,1.000014,0.005225,...,1.000000,0.688916,3.320373,1.000000,0.688916,3.320373,8.753887e-12,36627.0,2.523293e+04,1.216153e+05
education,float64,36627,36627,0,-1.753575,1.311478,-1.893384e-16,1.000027,1.000014,0.005225,...,1.000000,-0.154783,2.304695,1.000000,-0.154783,2.304695,-6.934897e-12,36627.0,-5.669249e+03,8.441408e+04
default,float64,36627,36627,0,-0.134309,7.445539,1.008770e-17,1.000027,1.000014,0.005225,...,1.000000,7.311231,54.454094,1.000000,7.311231,54.454094,3.694822e-13,36627.0,2.677884e+05,1.994490e+06
balance,float64,36627,36627,0,-2.727835,32.261864,-7.759770e-18,1.000027,1.000014,0.005225,...,1.000000,7.899990,124.863907,1.000000,7.899990,124.863907,-2.842171e-13,36627.0,2.893530e+05,4.573390e+06
housing,float64,36627,36627,0,-1.120923,0.892122,-5.587034e-17,1.000027,1.000014,0.005225,...,1.000000,-0.228801,1.052350,1.000000,-0.228801,1.052350,-2.046363e-12,36627.0,-8.380306e+03,3.854443e+04
loan,float64,36627,36627,0,-0.435047,2.298604,-3.996282e-17,1.000027,1.000014,0.005225,...,1.000000,1.863558,4.472848,1.000000,1.863558,4.472848,-1.463718e-12,36627.0,6.825653e+04,1.638270e+05
job_admin.,float64,36627,36627,0,-0.358292,2.791017,6.285414e-17,1.000027,1.000014,0.005225,...,1.000000,2.432725,6.918150,1.000000,2.432725,6.918150,2.302158e-12,36627.0,8.910341e+04,2.533911e+05
job_blue-collar,float64,36627,36627,0,-0.524872,1.905225,-4.345471e-17,1.000027,1.000014,0.005225,...,1.000000,1.380353,2.905373,1.000000,1.380353,2.905373,-1.591616e-12,36627.0,5.055818e+04,1.064151e+05
job_entrepreneur,float64,36627,36627,0,-0.184203,5.428786,-5.839227e-17,1.000027,1.000014,0.005225,...,1.000000,5.244582,28.505645,1.000000,5.244582,28.505645,-2.138734e-12,36627.0,1.920933e+05,1.044076e+06
job_housemaid,float64,36627,36627,0,-0.166760,5.996635,-1.241563e-17,1.000027,1.000014,0.005225,...,1.000000,5.829875,34.987446,1.000000,5.829875,34.987446,-4.547474e-13,36627.0,2.135308e+05,1.281485e+06


In [16]:
pd.reset_option('display.max_rows')


#### 安全性讨论

请注意，全表统计会暴露数据整体统计结果，其背后实际上蕴含了**sf.reveal**,请谨慎使用。

### 相关系数矩阵

我们接下来计算特征和特征之间，特征和标签之间的相关系数矩阵。

> 计算相关系数矩阵时，one-hot编码各列无需参与计算。

In [17]:
from secretflow.stats.ss_pearsonr_v import PearsonR

pearson_r_calculator = PearsonR(spu)
corr_matrix = pearson_r_calculator.pearsonr(vdf_hat)

import numpy as np

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
corr_matrix


(_run pid=8491) /home/zhanggong/anaconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=8491)   warnings.warn(
(_run pid=8495) /home/zhanggong/anaconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=8495)   warnings.warn(
(_run pid=8489) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=8489) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=8489) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
(_run pid=8489) INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Com

(_run pid=8495) [2023-06-17 19:42:38.775] [info] [thread_pool.cc:30] Create a fixed thread pool with size 7
(_run pid=8489) [2023-06-17 19:42:38.791] [info] [thread_pool.cc:30] Create a fixed thread pool with size 7


array([[1.000, -0.165, -0.019, 0.097, -0.187, -0.014, -0.009, 0.006,
        -0.025, -0.000, 0.026],
       [-0.165, 1.000, -0.010, 0.065, -0.075, -0.024, -0.001, 0.004,
        0.005, 0.024, 0.066],
       [-0.019, -0.010, 1.000, -0.066, -0.008, 0.075, -0.009, 0.020,
        -0.030, -0.017, -0.020],
       [0.097, 0.065, -0.066, 1.000, -0.070, -0.086, 0.019, -0.013,
        0.006, 0.019, 0.053],
       [-0.187, -0.075, -0.008, -0.070, 1.000, 0.039, -0.003, -0.023,
        0.124, 0.034, -0.143],
       [-0.014, -0.024, 0.075, -0.086, 0.039, 1.000, -0.008, 0.013,
        -0.019, -0.009, -0.065],
       [-0.009, -0.001, -0.009, 0.019, -0.003, -0.008, 1.000, -0.201,
        0.012, 0.005, 0.326],
       [0.006, 0.004, 0.020, -0.013, -0.023, 0.013, -0.201, 1.000,
        -0.088, -0.031, -0.070],
       [-0.025, 0.005, -0.030, 0.006, 0.124, -0.019, 0.012, -0.088,
        1.000, 0.441, 0.101],
       [-0.000, 0.024, -0.017, 0.019, 0.034, -0.009, 0.005, -0.031,
        0.441, 1.000, 0.087],
  

#### 安全性讨论

相关系数矩阵的计算需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

### VIF指标计算

隐语还支持VIF的计算来进行多重共线性检验。

> 计算VIF指标时，one-hot编码各列无需参与计算。

In [18]:
from secretflow.stats.ss_vif_v import VIF

vif_calculator = VIF(spu)
vif_results = vif_calculator.vif(vdf_hat)
print(vdf_hat.columns)
print(vif_results)


(_run pid=8495) /home/zhanggong/anaconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=8495)   warnings.warn(
(_run pid=8489) /home/zhanggong/anaconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=8489)   warnings.warn(


Index(['age', 'education', 'default', 'balance', 'housing', 'loan', 'duration',
       'campaign', 'pdays', 'previous', 'y'],
      dtype='object')
[1.083 1.051 1.011 1.030 1.093 1.017 1.165 1.051 1.280 1.246 1.173]


#### 安全性讨论

VIF指标的计算需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

## 模型训练

接下来，我们将会分别训练一个逻辑回归模型和一个XGB模型。


### 随机分割

在训练之前，我们需要将数据分割为训练集和验证集。

其中train_x和train_y为训练集的特征和标签。test_x和test_y为训练集的特征和标签。


In [19]:
from secretflow.data.split import train_test_split

random_state = 1234

train_vdf, test_vdf = train_test_split(vdf, train_size=0.8, random_state=random_state)

train_x = train_vdf.drop(columns=['y'])
train_y = train_vdf['y']

test_x = test_vdf.drop(columns=['y'])
test_y = test_vdf['y']


#### 安全性讨论

随机分割时，每一方会共享随机数种子，并由每一方数据的owner分别执行各自的数据分割并且确保最终分割结果仍然是对齐的。

### PSI（人群稳定性分析）

样本稳定指数是衡量样本变化所产生的偏移量的一种重要指标，通常用来衡量样本的稳定程度，比如样本在两个月份之间的变化是否稳定。通常变量的PSI值在0.1以下表示变化不太显著，在0.1到0.25之间表示有比较显著的变化，大于0.25表示变量变化比较剧烈，需要特殊关注。

接下来以`balance`为例子，确认两次抽样的样本分布是否接近。

> 根据业务需求，PSI分析也可以在数据分析或者特征预处理的时候进行。


In [20]:
stats_df = table_statistics(train_x['balance'])

In [21]:
min_val, max_val = stats_df['min'], stats_df['max']

In [22]:
from secretflow.stats import psi_eval
from secretflow.stats.core.utils import equal_range
import jax.numpy as jnp

split_points = equal_range(jnp.array([min_val, max_val]), 3)
balance_psi_score = psi_eval(train_x['balance'], test_x['balance'], split_points)

sf.reveal(balance_psi_score)

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.


Array(inf, dtype=float32)

#### 安全性讨论

PSI分析是一个单方运算，由数据owner的PYU Device执行计算。

### 逻辑回归模型

使用 **ml.linear.ss_sgd.SSRegression** 可以进行密态逻辑回归模型的训练。

请参考相关的API文档。


In [23]:
from secretflow.ml.linear.ss_sgd import SSRegression

lr_model = SSRegression(spu)
lr_model.fit(
    x=train_x,
    y=train_y,
    epochs=3,
    learning_rate=0.1,
    batch_size=128,
    sig_type='t1',
    reg_type='logistic',
    penalty='l2',
    l2_norm=0.5,
)


INFO:root:epoch 1 times: 10.901921272277832s
INFO:root:epoch 2 times: 3.902287244796753s
INFO:root:epoch 3 times: 2.805579423904419s


你可能会对为何上面的语句很快就执行完毕感到困惑，原因是在隐语中，语句都是lazy evaluation的，在上面的例子中，直到lr_model被真正被使用的时候，**lr_model.fit**才会被执行。

#### 安全性讨论

SSRegression的训练基于SPU Device，双方的原始数据将会被保护。

### XGBoost模型

使用 **ml.boost.ss_xgb_v.Xgb** 可以进行密态XGBoost模型的训练。

请参考相关的API文档。

In [24]:
from secretflow.ml.boost.ss_xgb_v import Xgb

xgb = Xgb(spu)
params = {
    'num_boost_round': 3,
    'max_depth': 5,
    'sketch_eps': 0.25,
    'objective': 'logistic',
    'reg_lambda': 0.2,
    'subsample': 1,
    'colsample_by_tree': 1,
    'base_score': 0.5,
}
xgb_model = xgb.train(params=params, dtrain=train_x, label=train_y)


INFO:root:Create proxy actor <class 'secretflow.ml.boost.ss_xgb_v.core.tree_worker.XgbTreeWorker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.boost.ss_xgb_v.core.tree_worker.XgbTreeWorker'> with party bob.
INFO:root:fragment_count 1
INFO:root:prepare time 0.5018086433410645s
INFO:root:global_setup time 4.407729864120483s
INFO:root:build & infeed bucket_map fragments [0, 0]
INFO:root:build & infeed bucket_map time 1.0847680568695068s
INFO:root:init_pred time 0.06759953498840332s
INFO:root:epoch 0 tree_setup time 0.14282703399658203s
INFO:root:fragment[0, 0] gradient sum time 1.2693753242492676s
INFO:root:level 0 time 1.8847157955169678s
INFO:root:fragment[0, 0] gradient sum time 1.426650047302246s
INFO:root:level 1 time 2.1389851570129395s
INFO:root:fragment[0, 0] gradient sum time 3.193174123764038s
INFO:root:level 2 time 3.9200615882873535s
INFO:root:fragment[0, 0] gradient sum time 5.8141584396362305s
INFO:root:level 3 time 6.5092995166778564s
INFO:root:frag

(_run pid=8491) [2023-06-17 19:45:11.196] [info] [thread_pool.cc:30] Create a fixed thread pool with size 7
(XgbTreeWorker pid=14236) [2023-06-17 19:45:11.125] [info] [thread_pool.cc:30] Create a fixed thread pool with size 7
(XgbTreeWorker pid=14237) [2023-06-17 19:45:11.157] [info] [thread_pool.cc:30] Create a fixed thread pool with size 7


INFO:root:epoch 0 time 30.85298466682434s
INFO:root:epoch 1 tree_setup time 0.8186788558959961s
INFO:root:fragment[0, 0] gradient sum time 2.40937876701355s
INFO:root:level 0 time 3.2553441524505615s
INFO:root:fragment[0, 0] gradient sum time 2.408903121948242s
INFO:root:level 1 time 3.131530523300171s
INFO:root:fragment[0, 0] gradient sum time 3.6030757427215576s
INFO:root:level 2 time 4.206820964813232s
INFO:root:fragment[0, 0] gradient sum time 6.732248067855835s
INFO:root:level 3 time 7.682126760482788s
INFO:root:fragment[0, 0] gradient sum time 13.995256185531616s
INFO:root:level 4 time 15.290855169296265s
INFO:root:epoch 1 time 34.85509014129639s
INFO:root:epoch 2 tree_setup time 0.808718204498291s
INFO:root:fragment[0, 0] gradient sum time 1.9684031009674072s
INFO:root:level 0 time 2.44930100440979s
INFO:root:fragment[0, 0] gradient sum time 1.735743761062622s
INFO:root:level 1 time 2.647735834121704s
INFO:root:fragment[0, 0] gradient sum time 4.358854293823242s
INFO:root:level 

Xgb.train将会直接执行，请耐心等待。

#### 安全性讨论

Xgb的训练基于SPU Device，双方的原始数据将会被保护。

## 模型预测

接下来，我们将会分别利用刚刚训练好的模型来预测测试集。

### 逻辑回归模型

由于在我们的场景下，数据集标签的持有者是bob，因此我们在这里将预测结果**reveal**给bob.

In [25]:
lr_y_hat = lr_model.predict(x=test_x, batch_size=1024, to_pyu=bob)

#### 安全性讨论

逻辑回归的预测基于SPU Device，双方的原始数据将会被保护。

当设置**to_pyu**，预测结果将会被reveal给该方，否则将仍然保持秘密分享的状态。

### XGBoost模型

由于在我们的场景下，数据集标签的持有者是bob，因此我们在这里将预测结果**reveal**给bob.

In [26]:
xgb_y_hat = xgb_model.predict(dtrain=test_x, to_pyu=bob)


INFO:root:Create proxy actor <class 'secretflow.ml.boost.ss_xgb_v.core.tree_worker.XgbTreeWorker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.boost.ss_xgb_v.core.tree_worker.XgbTreeWorker'> with party bob.


(XgbTreeWorker pid=14665) [2023-06-17 19:48:37.547] [info] [thread_pool.cc:30] Create a fixed thread pool with size 7
(XgbTreeWorker pid=14666) [2023-06-17 19:48:37.597] [info] [thread_pool.cc:30] Create a fixed thread pool with size 7


#### 安全性讨论

XGBoost模型的预测基于SPU Device，双方的原始数据将会被保护。

当设置**to_pyu**，预测结果将会被reveal给该方，否则将仍然保持秘密分享的状态。

## 模型评估

接下来，我们将利用测试数据集对模型效果进行评估，包括：

- 二分类评估
- PVA
- P-Value
- 评分卡转换

### 二分类评估

隐语中对二分类的评估有集成的支持。

`BiClassificationEval` 将计算 `AUC`, `KS`, `F1 Score`, `Lift`, `K-S`, `Gain`, `Precision`, `Recall` 等统计数值， 并提供（基于prediction score的）等频和等距分箱的统计报告和总报告。

不同分桶中评估模型的预测的`threshold`不同。总报告中依赖`threshold`的统计取的是各个分桶的最佳值。

详情可以参考API文档。

In [27]:
from secretflow.stats.biclassification_eval import BiClassificationEval

biclassification_evaluator = BiClassificationEval(
    y_true=test_y, y_score=lr_y_hat, bucket_size=20
)
lr_report = sf.reveal(biclassification_evaluator.get_all_reports())


In [28]:
print(f'positive_samples: {lr_report.summary_report.positive_samples}')
print(f'negative_samples: {lr_report.summary_report.negative_samples}')
print(f'total_samples: {lr_report.summary_report.total_samples}')
print(f'auc: {lr_report.summary_report.auc}')
print(f'ks: {lr_report.summary_report.ks}')
print(f'f1_score: {lr_report.summary_report.f1_score}')


positive_samples: 838.0
negative_samples: 6488.0
total_samples: 7326.0
auc: 0.9131353497505188
ks: 0.6892967820167542
f1_score: 0.5826446413993835


In [29]:
biclassification_evaluator = BiClassificationEval(
    y_true=test_y, y_score=xgb_y_hat, bucket_size=20
)
xgb_report = sf.reveal(biclassification_evaluator.get_all_reports())


In [30]:
print(f'positive_samples: {xgb_report.summary_report.positive_samples}')
print(f'negative_samples: {xgb_report.summary_report.negative_samples}')
print(f'total_samples: {xgb_report.summary_report.total_samples}')
print(f'auc: {xgb_report.summary_report.auc}')
print(f'ks: {xgb_report.summary_report.ks}')
print(f'f1_score: {xgb_report.summary_report.f1_score}')


positive_samples: 838.0
negative_samples: 6488.0
total_samples: 7326.0
auc: 0.8850927352905273
ks: 0.6121011972427368
f1_score: 0.5305185914039612


### PVA (预测和实际平均值比较)

结果由`abs(mean(Acutal) - mean(Prediction))`计算获得, 值越小越好。

In [31]:
from secretflow.stats import pva_eval

lr_pva_score = pva_eval(test_y, lr_y_hat, 1)

sf.reveal(lr_pva_score)


In [32]:
xgb_pva_score = pva_eval(test_y, xgb_y_hat, 1)

sf.reveal(xgb_pva_score)


### P-Value
双方可通过p-value的值来判断参数是否显著，即该自变量是否可以有效预测因变量的变异, 从而判定对应的解释变量是否应包括在模型中。


In [33]:
from secretflow.stats import SSPValue

model = lr_model.save_model()
sspv = SSPValue(spu)
pvalues = sspv.pvalues(test_x, test_y, model)

pvalues

### 评分卡转换

> 严格来说，评分卡转化是对预测结果的后续处理，并不属于模型评估。


我们将 `y = 1` 的概率设为`p`， `odds = p / (1 - p)`, 评分卡设定的分值刻度可以通过将分值表示为比率对数的线性表达式来定义，即可表示为下式：

`Score = A - B log(odds)`， A 和 B 是可以设定的常数。隐语中提供了评分卡转换功能，详情可以参考API文档。

In [34]:
from secretflow.stats import BiClassificationEval, ScoreCard

sc = ScoreCard(20, 600, 20)
score = sc.transform(xgb_y_hat)

sf.reveal(score.partitions[bob])


### 安全性讨论

以上所有模型评估的方法均为单方运算，由label拥有者的PYU Device进行运算。

## 实验结束

最后，我们需要清理临时文件，并关闭隐语cluster。

In [35]:
import os

try:
    os.remove(alice_path)
    os.remove(alice_psi_path)
    os.remove(bob_path)
    os.remove(bob_psi_path)
except OSError:
    pass

sf.shutdown()

恭喜！你已经完成了隐语金融风控全链路的全部实验内容。

如果你对本实验有任何建议和问题，请在[Github Issues](https://github.com/secretflow/secretflow/issues)上联系我们。